## Installation

    !conda install -c conda-forge --yes glpk
    !conda install -c conda-forge --yes pyomo

In [66]:
from pyomo.environ import *
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x + 30*model.y,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
SolverFactory('glpk').solve(model)

# display solution
print("Profit = ", model.profit())
print("Units of X = ", model.x())
print("Units of Y = ", model.y())

print("")
model.pprint()

Profit =  2600.0
Units of X =  20.0
Units of Y =  60.0

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  20.0 :  None : False : False : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x + 30*y

3 Constraint Declarations
    demand : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :  40.0 :   True
    laborA : Size=1, Index=None, Active=True
        Key  : Lower : Body  : Upper : Active
        None :  -Inf : x + y :  80.0 :   True
    laborB : Size=1, Index=None, Active=True
        Key  : Lower : Body    : Upper : Active
        None :  -Inf : 2*x + y : 100.0 :   True

In [68]:
from pyomo.environ import *
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x <= 80)
model.laborB = Constraint(expr = 2*model.x <= 100)

# solve
SolverFactory('glpk').solve(model)

# display solution
print("Profit = ", model.profit())
print("Units of X = ", model.x())

Profit =  1600.0
Units of X =  40.0


### Product Information

In [62]:
PRODUCTS = ['X','Y']

product_profit = {}
product_profit['X'] = 40
product_profit['Y'] = 30

product_demand = {}
product_demand['X'] = 40
product_demand['Y'] = 10000

### Labor Information

In [63]:
RESOURCES = ['Labor A','Labor B']

resource_available = {}
resource_available['Labor A'] = 80
resource_available['Labor B'] = 100

resource_used = {}
resource_used['Labor A','X'] = 1
resource_used['Labor A','Y'] = 1
resource_used['Labor B','X'] = 2
resource_used['Labor B','Y'] = 1

In [65]:
from pyomo.environ import *
model = ConcreteModel()

# creates a variable associated with each product
model.x = Var(PRODUCTS,domain=NonNegativeReals)

# create an objective function
model.Profit = Objective(
    expr = sum([product_profit[p]*model.x[p] for p in PRODUCTS]),
    sense = maximize)

# add list of demand constraints
model.Demand = ConstraintList()
for p in PRODUCTS:
    model.Demand.add(expr = model.x[p] <= product_demand[p])

# add list of labor constraints
model.resource = ConstraintList()
for r in RESOURCES:
    model.resource.add(sum([resource_used[r,p]*model.x[p] for p in PRODUCTS]) <= resource_available[r])

SolverFactory('glpk').solve(model)
model.pprint()

3 Set Declarations
    Demand_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        [1, 2]
    resource_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        [1, 2]
    x_index : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=False, Bounds=None
        ['X', 'Y']

1 Var Declarations
    x : Size=2, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          X :     0 :  20.0 :  None : False : False : NonNegativeReals
          Y :     0 :  60.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    Profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*x[X] + 30*x[Y]

2 Constraint Declarations
    Demand : Size=2, Index=Demand_index, Active=True
        Key : Lower : Body : Upper   : Active
          1 :  -Inf : x[X] :    40.0 :   True
          2 :  -Inf : x[Y] : 10000.0 :   True
    resource : Size=2, Index=resource_index, 

In [31]:
for p in PRODUCTS:
    print(p,model.x[p]())

Product X 20.0
Widget Y 60.0
